In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Here, I have taken the dataseet from Kaggkle. Its a CSV file. here, is the link of the dataseet->https://www.kaggle.com/datasets/oddrationale/mnist-in-csv/data

In [21]:
data=pd.read_csv("/content/sample_data/mnist_test.csv")
data.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Pre-processing

In [22]:
data.shape

(9999, 785)

In [23]:
data.isnull().sum() #NoNull values in this dataset

7        0
0        0
0.1      0
0.2      0
0.3      0
        ..
0.663    0
0.664    0
0.665    0
0.666    0
0.667    0
Length: 785, dtype: int64

In [24]:
data.describe()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
count,9999.000000,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0,...,9999.000000,9999.000000,9999.000000,9999.000000,9999.0,9999.0,9999.0,9999.0,9999.0,9999.0
mean,4.443144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179318,0.163616,0.052605,0.000600,0.0,0.0,0.0,0.0,0.0,0.0
std,2.895897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.674433,5.736359,2.420125,0.060003,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,7.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,9.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,253.000000,253.000000,156.000000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Split data into training and testing sets
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_flat, y_train, test_size=0.2, random_state=42)


#ANN Implementation from scratch

In [26]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros((1, self.output_size))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        exp_scores = np.exp(self.z2)
        self.probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        return self.probs

    def backward(self, X, y, learning_rate):
        num_examples = X.shape[0]
        delta3 = self.probs
        delta3[range(num_examples), y] -= 1
        dW2 = np.dot(self.a1.T, delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = np.dot(delta3, self.W2.T) * (self.a1 * (1 - self.a1))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2

    def fit(self, X, y, learning_rate=0.0002, epochs=1000):
        for i in range(epochs):
            self.forward(X)
            self.backward(X, y, learning_rate)

    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)


#ANN using Keras

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train_split, y_train_split, epochs=10, validation_data=(x_val_split, y_val_split))


Epoch 1/10
1500/1500 [==============================] - 9s 5ms/step - loss: 0.2873 - accuracy: 0.9176 - val_loss: 0.1610 - val_accuracy: 0.9549
Epoch 2/10
1500/1500 [==============================] - 7s 5ms/step - loss: 0.1298 - accuracy: 0.9621 - val_loss: 0.1247 - val_accuracy: 0.9639
Epoch 3/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0883 - accuracy: 0.9741 - val_loss: 0.1017 - val_accuracy: 0.9696
Epoch 4/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0661 - accuracy: 0.9804 - val_loss: 0.0910 - val_accuracy: 0.9720
Epoch 5/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.0497 - accuracy: 0.9850 - val_loss: 0.0857 - val_accuracy: 0.9752
Epoch 6/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0390 - accuracy: 0.9884 - val_loss: 0.0919 - val_accuracy: 0.9732
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0304 - accuracy: 0.9905 - val_loss: 0.0899 - val_accurac

In [29]:
# Instantiate custom ANN
custom_ann = NeuralNetwork(input_size=784, hidden_size=128, output_size=10)

# Fit custom ANN
custom_ann.fit(x_train_split, y_train_split)

# Evaluate custom ANN
custom_ann_accuracy = np.mean(custom_ann.predict(x_test_flat) == y_test) * 100

# Evaluate Keras model
_, keras_accuracy = model.evaluate(x_test_flat, y_test)

print("Custom ANN Accuracy:", custom_ann_accuracy)



313/313 [==============================] - 1s 3ms/step - loss: 0.0824 - accuracy: 0.9755
Custom ANN Accuracy: 94.37
Keras Accuracy: 0.9754999876022339


In [30]:
print("Keras Accuracy:", keras_accuracy*100)

Keras Accuracy: 97.54999876022339


# Scratch !

In [31]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [32]:
Y_train

array([1, 4, 2, ..., 6, 6, 0])

In [49]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    m = Y.shape[0]
    dZ2 = A2 - one_hot_Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [50]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [51]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.80, 500)

Iteration:  0
[4 0 0 ... 0 7 6] [1 4 2 ... 6 6 0]
0.10223358150905656
Iteration:  10
[1 9 2 ... 1 0 8] [1 4 2 ... 6 6 0]
0.5513945993999333
Iteration:  20
[1 4 7 ... 1 0 8] [1 4 2 ... 6 6 0]
0.6738526502944772
Iteration:  30
[1 4 7 ... 1 6 6] [1 4 2 ... 6 6 0]
0.6838537615290587
Iteration:  40
[1 4 7 ... 6 6 5] [1 4 2 ... 6 6 0]
0.7805311701300145
Iteration:  50
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8264251583509279
Iteration:  60
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8469829981109012
Iteration:  70
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8647627514168241
Iteration:  80
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8677630847871985
Iteration:  90
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8798755417268586
Iteration:  100
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8850983442604734
Iteration:  110
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8882098010890099
Iteration:  120
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8898766529614401
Iteration:  130
[1 4 7 ... 6 6 0] [1 4 2 ... 6 6 0]
0.8957661962440271
Iteration:  140


In [52]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [53]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[8 4 8 6 6 1 7 3 1 8 3 6 6 1 1 1 8 2 6 8 2 6 1 7 3 0 6 3 4 7 3 6 3 1 5 3 7
 7 5 4 3 8 7 3 2 3 8 8 9 6 7 7 3 5 5 7 8 1 0 8 1 8 8 1 7 2 1 2 7 3 0 1 7 0
 3 5 3 1 7 3 7 9 5 5 3 2 9 1 9 2 6 3 1 9 6 4 1 4 5 2 0 8 9 6 3 7 2 2 7 5 3
 7 9 6 3 0 2 9 3 6 4 0 5 0 3 8 8 4 7 8 9 0 4 8 6 3 1 8 7 5 8 5 1 3 6 9 5 4
 7 0 8 0 5 4 1 9 5 0 5 3 2 7 0 7 4 5 1 8 9 6 2 1 5 4 1 7 2 3 5 3 7 7 2 4 9
 1 8 3 0 1 0 3 5 1 9 7 3 6 1 8 9 2 1 1 0 2 2 5 3 2 5 4 0 9 5 4 3 0 0 7 2 3
 1 6 9 0 8 1 6 8 2 0 0 3 1 5 2 9 0 3 5 3 4 0 1 7 6 6 8 7 8 9 8 9 1 0 1 6 7
 7 7 1 4 0 4 5 5 9 2 1 9 2 5 0 7 4 4 5 9 6 3 4 2 3 0 8 3 4 5 2 3 4 4 3 4 9
 0 6 1 7 3 2 5 0 7 5 4 0 5 5 4 4 7 0 2 2 8 6 6 7 4 9 2 2 6 4 7 6 0 8 1 9 4
 2 6 8 0 1 4 7 2 3 6 2 3 8 5 5 8 8 9 7 5 1 4 3 3 3 8 8 0 4 0 9 1 5 6 5 9 7
 0 4 2 6 8 1 5 8 7 9 3 8 5 0 6 7 1 6 1 0 2 9 0 4 7 8 0 3 7 2 7 2 5 4 3 6 0
 7 0 5 9 4 4 1 6 5 6 9 2 1 0 7 9 2 7 7 6 6 8 5 4 1 4 2 4 3 1 3 0 1 6 1 2 9
 9 9 0 2 5 1 9 7 7 1 9 7 0 5 8 2 1 8 4 4 2 2 3 6 7 1 6 2 8 3 0 6 4 8 5 6 7
 2 8 0 4 3 9 3 9 7 6 8 3 

0.902